# Exploratory Data Analysis on RAW data

## Modules

In [1]:
import pandas as pd
import json
import fnmatch
import os
import re

## Variables

In [2]:
PATH_FOLDER_READ='01.Raw data/'
PATH_FOLDER_WRITE='02.Clean data/'

## Functions

In [3]:
def cleanNotes(s):
    s=re.sub(r"\[(.*?)\]",'',s)
    s.strip()
    return s

In [4]:
def cleanDates(s):
    s=re.search(r'\((.*?)\)',s).group(1)
    s.strip()
    return s

## Loop all Json file of folder

In [5]:
df_c = pd.DataFrame()

In [6]:
for file in os.listdir(PATH_FOLDER_READ):
    if fnmatch.fnmatch(file.upper(), '*.JSON'):
        print(file)
        
        # Read file
        with open(PATH_FOLDER_READ+file, 'r') as fp:
            data = json.load(fp)
        
        # Transform data into a DF
        df = pd.DataFrame.from_dict(data[0])

        # Headr on first row (ref https://stackoverflow.com/a/31328974/1008588)
        new_header = df.iloc[0]
        df = df[1:]
        df.columns = new_header
        
        # Clean Wikipedia notes reference (clean up of anything between square brackets)
        df.rename(columns=lambda c: re.sub(r"\[(.*?)\]",'',c), inplace=True)
        
        # Strip spaces in columns header
        df.rename(columns=lambda c: c.strip(), inplace=True)
        
        # Add Season number as new column
        df['No. Season'] = int(file[14:16])
        
        # Concat with existing DF df_c
        df_c= pd.concat([df_c, df])
df=df_c

Episodes_RAW_S10.json
Episodes_RAW_S06.json
Episodes_RAW_S26.json
Episodes_RAW_S30.json
Episodes_RAW_S31.json
Episodes_RAW_S27.json
Episodes_RAW_S07.json
Episodes_RAW_S11.json
Episodes_RAW_S16.json
Episodes_RAW_S01.json
Episodes_RAW_S17.json
Episodes_RAW_S21.json
Episodes_RAW_S34.json
Episodes_RAW_S22.json
Episodes_RAW_S18.json
Episodes_RAW_S02.json
Episodes_RAW_S14.json
Episodes_RAW_S15.json
Episodes_RAW_S03.json
Episodes_RAW_S19.json
Episodes_RAW_S23.json
Episodes_RAW_S28.json
Episodes_RAW_S04.json
Episodes_RAW_S12.json
Episodes_RAW_S32.json
Episodes_RAW_S24.json
Episodes_RAW_S08.json
Episodes_RAW_S09.json
Episodes_RAW_S25.json
Episodes_RAW_S33.json
Episodes_RAW_S13.json
Episodes_RAW_S05.json
Episodes_RAW_S29.json


In [7]:
# reset index
df.reset_index(inplace = True, drop = True)

In [8]:
df.head()

,No.overall,No. inseason,Title,Directed by,Written by,Original air date,Prod.code,U.S. viewers(millions),No. Season,Screenplay by,Release date(U.S.)
0,204,1,"""Lard of the Dance""",Dominic Polcino,Jane O'Brien,"August 23, 1998 (1998-08-23)",5F20,7[210],10,NaN,NaN
1,205,2,"""The Wizard of Evergreen Terrace""",Mark Kirkland,John Swartzwelder,"September 20, 1998 (1998-09-20)",5F21,7.95[211],10,NaN,NaN
2,206,3,"""Bart the Mother""",Steven Dean Moore,David X. Cohen,"September 27, 1998 (1998-09-27)",5F22,7.35[212],10,NaN,NaN
3,207,4,"""Treehouse of Horror IX""",Steven Dean Moore,Donick CaryLarry DoyleDavid X. Cohen,"October 25, 1998 (1998-10-25)",AABF01,8.5[213],10,NaN,NaN
4,208,5,"""When You Dish Upon a Star""",Pete Michels,Richard Appel,"November 8, 1998 (1998-11-08)",5F19,9[214],10,NaN,NaN


## Manage specific cases

### Delete The Simpsons movie

In [9]:
df.loc[df['Title'] == 'The Simpsons Movie']

,No.overall,No. inseason,Title,Directed by,Written by,Original air date,Prod.code,U.S. viewers(millions),No. Season,Screenplay by,Release date(U.S.)
408,NaN,NaN,The Simpsons Movie,David Silverman,NaN,NaN,NaN,NaN,19,"James L. Brooks, Matt Groening, Al Jean, Ian M...","July 27, 2007 (2007-07-27)"


In [10]:
df = df.drop(df[df['Title'] == 'The Simpsons Movie'].index)

### Episodes made of two parts: 681 and 682

In [11]:
df.loc[df['No.overall'] == '681682']

,No.overall,No. inseason,Title,Directed by,Written by,Original air date,Prod.code,U.S. viewers(millions),No. Season,Screenplay by,Release date(U.S.)
111,681682,1920,"""Warrin' Priests"" (Parts One and Two)",Bob AndersonMatthew Nastuk,Pete Holmes,"April 26, 2020 (2020-04-26)May 3, 2020 (2020-0...",ZABF12ZABF13,1.35[286]1.36[287],31,NaN,NaN


In [12]:
df = df.drop(df[df['No.overall'] == '681682'].index)

In [13]:
newRow={'No.overall':'681',
    'No. inseason':'19',
    'Title':'"Warrin\' Priests" (Parts One)',
    'Directed by':'Bob Anderson',
    'Written by':'Pete Holmes',
    'Original air date':'April 26, 2020 (2020-04-26)',
    'Prod.code':'ZABF12',
    'U.S. viewers(millions)':'1.35[287]',
    'No. Season':'31',
    'Screenplay by':'NaN',
    'Release date(U.S.)':'NaN'}

In [14]:
dn=pd.DataFrame(newRow, index=[0])

In [15]:
df= pd.concat([df, dn])

In [16]:
newRow={'No.overall':'682',
    'No. inseason':'20',
    'Title':'"Warrin\' Priests" (Parts Two)',
    'Directed by':'Matthew Nastuk',
    'Written by':'Pete Holmes',
    'Original air date':'May 03, 2020 (2020-05-03)',
    'Prod.code':'ZABF13',
    'U.S. viewers(millions)':'1.36[288]',
    'No. Season':'31',
    'Screenplay by':'NaN',
    'Release date(U.S.)':'NaN'}

In [17]:
dn=pd.DataFrame(newRow, index=[0])

In [18]:
df= pd.concat([df, dn])

### Episodes made of two parts: 608 and 609

In [19]:
df.loc[df['No.overall'] == '608609']

,No.overall,No. inseason,Title,Directed by,Written by,Original air date,Prod.code,U.S. viewers(millions),No. Season,Screenplay by,Release date(U.S.)
442,608609,1213,"""The Great Phatsby"" (Vol. I & Vol. II)",Chris ClementsTimothy Bailey,Dan GreaneyMatt Selman (part 2 only),"January 15, 2017 (2017-01-15)",WABF04WABF05,6.90[214],28,NaN,NaN


In [20]:
df = df.drop(df[df['No.overall'] == '608609'].index)

In [21]:
newRow={'No.overall':'608',
    'No. inseason':'12',
    'Title':'"The Great Phatsby" (Vol. I)',
    'Directed by':'Chris Clements',
    'Written by':'Dan Greaney',
    'Original air date':'January 15, 2017 (2017-01-15)',
    'Prod.code':'WABF04',
    'U.S. viewers(millions)':'6.90[214]',
    'No. Season':'28',
    'Screenplay by':'NaN',
    'Release date(U.S.)':'NaN'}

In [22]:
dn=pd.DataFrame(newRow, index=[0])

In [23]:
df= pd.concat([df, dn])

In [24]:
newRow={'No.overall':'609',
    'No. inseason':'12',
    'Title':'"The Great Phatsby" (Vol. II)',
    'Directed by':'Timothy Bailey',
    'Written by':'Matt Selman',
    'Original air date':'January 15, 2017 (2017-01-15)',
    'Prod.code':'WABF04',
    'U.S. viewers(millions)':'6.90[214]',
    'No. Season':'28',
    'Screenplay by':'NaN',
    'Release date(U.S.)':'NaN'}

In [25]:
dn=pd.DataFrame(newRow, index=[0])

In [26]:
df= pd.concat([df, dn])

### Episodes made of two parts: 712 and 713

In [27]:
df.loc[df['No.overall'] == '712713']

,No.overall,No. inseason,Title,Directed by,Written by,Original air date,Prod.code,U.S. viewers(millions),No. Season,Screenplay by,Release date(U.S.)
616,712713,67,"""A Serious Flanders"" (Parts 1 & 2)",Debbie Bruce MahanMatthew Faughnan,Cesar Mazariegos,"November 7, 2021 (2021-11-07)November 14, 2021...",QABF21QABF22,3.47[320]1.66[321],33,NaN,NaN


In [28]:
df = df.drop(df[df['No.overall'] == '712713'].index)

In [29]:
newRow={'No.overall':'712',
    'No. inseason':'6',
    'Title':'"A Serious Flanders" (Parts 1)"',
    'Directed by':'Debbie Bruce Mahan',
    'Written by':'Cesar Mazariegos',
    'Original air date':'November 7, 2021 (2021-11-07)',
    'Prod.code':'QABF21',
    'U.S. viewers(millions)':'3.47[320]',
    'No. Season':'33',
    'Screenplay by':'NaN',
    'Release date(U.S.)':'NaN'}

In [30]:
dn=pd.DataFrame(newRow, index=[0])

In [31]:
df= pd.concat([df, dn])

In [32]:
newRow={'No.overall':'713',
    'No. inseason':'7',
    'Title':'"A Serious Flanders" (Parts 2)"',
    'Directed by':'Matthew Faughnan',
    'Written by':'Cesar Mazariegos',
    'Original air date':'November 14, 2021 (2021-11-14)',
    'Prod.code':'QABF22',
    'U.S. viewers(millions)':'1.66[321]',
    'No. Season':'33',
    'Screenplay by':'NaN',
    'Release date(U.S.)':'NaN'}

In [33]:
dn=pd.DataFrame(newRow, index=[0])

In [34]:
df= pd.concat([df, dn])

In [35]:
# reset index
df.reset_index(inplace = True, drop = True)

## Manage Wikipedia notes inside cells

In [36]:
df.head()

,No.overall,No. inseason,Title,Directed by,Written by,Original air date,Prod.code,U.S. viewers(millions),No. Season,Screenplay by,Release date(U.S.)
0,204,1,"""Lard of the Dance""",Dominic Polcino,Jane O'Brien,"August 23, 1998 (1998-08-23)",5F20,7[210],10,NaN,NaN
1,205,2,"""The Wizard of Evergreen Terrace""",Mark Kirkland,John Swartzwelder,"September 20, 1998 (1998-09-20)",5F21,7.95[211],10,NaN,NaN
2,206,3,"""Bart the Mother""",Steven Dean Moore,David X. Cohen,"September 27, 1998 (1998-09-27)",5F22,7.35[212],10,NaN,NaN
3,207,4,"""Treehouse of Horror IX""",Steven Dean Moore,Donick CaryLarry DoyleDavid X. Cohen,"October 25, 1998 (1998-10-25)",AABF01,8.5[213],10,NaN,NaN
4,208,5,"""When You Dish Upon a Star""",Pete Michels,Richard Appel,"November 8, 1998 (1998-11-08)",5F19,9[214],10,NaN,NaN


In [37]:
df['U.S. viewers(millions)']=df['U.S. viewers(millions)'].apply(str)
df['U.S. viewers(millions)']=df['U.S. viewers(millions)'].apply(cleanNotes)

In [38]:
df.head()

,No.overall,No. inseason,Title,Directed by,Written by,Original air date,Prod.code,U.S. viewers(millions),No. Season,Screenplay by,Release date(U.S.)
0,204,1,"""Lard of the Dance""",Dominic Polcino,Jane O'Brien,"August 23, 1998 (1998-08-23)",5F20,7,10,NaN,NaN
1,205,2,"""The Wizard of Evergreen Terrace""",Mark Kirkland,John Swartzwelder,"September 20, 1998 (1998-09-20)",5F21,7.95,10,NaN,NaN
2,206,3,"""Bart the Mother""",Steven Dean Moore,David X. Cohen,"September 27, 1998 (1998-09-27)",5F22,7.35,10,NaN,NaN
3,207,4,"""Treehouse of Horror IX""",Steven Dean Moore,Donick CaryLarry DoyleDavid X. Cohen,"October 25, 1998 (1998-10-25)",AABF01,8.5,10,NaN,NaN
4,208,5,"""When You Dish Upon a Star""",Pete Michels,Richard Appel,"November 8, 1998 (1998-11-08)",5F19,9,10,NaN,NaN


In [39]:
df['Original air date']=df['Original air date'].apply(str)
df['Original air date']=df['Original air date'].apply(cleanDates)

## Fix datatype of numeric columns

In [40]:
df['No.overall'] = pd.to_numeric(df['No.overall'])
df['No. inseason'] = pd.to_numeric(df['No. inseason'])
df['No. Season'] = pd.to_numeric(df['No. Season'])

df['U.S. viewers(millions)'] = df['U.S. viewers(millions)'].replace(['TBD'], None)
df['U.S. viewers(millions)'] = df['U.S. viewers(millions)'].replace(['N/A'], None)
df['U.S. viewers(millions)'] = pd.to_numeric(df['U.S. viewers(millions)'])

## Final check

In [41]:
df.head()

,No.overall,No. inseason,Title,Directed by,Written by,Original air date,Prod.code,U.S. viewers(millions),No. Season,Screenplay by,Release date(U.S.)
0,204,1,"""Lard of the Dance""",Dominic Polcino,Jane O'Brien,1998-08-23,5F20,7.00,10,NaN,NaN
1,205,2,"""The Wizard of Evergreen Terrace""",Mark Kirkland,John Swartzwelder,1998-09-20,5F21,7.95,10,NaN,NaN
2,206,3,"""Bart the Mother""",Steven Dean Moore,David X. Cohen,1998-09-27,5F22,7.35,10,NaN,NaN
3,207,4,"""Treehouse of Horror IX""",Steven Dean Moore,Donick CaryLarry DoyleDavid X. Cohen,1998-10-25,AABF01,8.50,10,NaN,NaN
4,208,5,"""When You Dish Upon a Star""",Pete Michels,Richard Appel,1998-11-08,5F19,9.00,10,NaN,NaN


In [42]:
df.tail()

,No.overall,No. inseason,Title,Directed by,Written by,Original air date,Prod.code,U.S. viewers(millions),No. Season,Screenplay by,Release date(U.S.)
694,682,20,"""Warrin' Priests"" (Parts Two)",Matthew Nastuk,Pete Holmes,2020-05-03,ZABF13,1.36,31,NaN,NaN
695,608,12,"""The Great Phatsby"" (Vol. I)",Chris Clements,Dan Greaney,2017-01-15,WABF04,6.90,28,NaN,NaN
696,609,12,"""The Great Phatsby"" (Vol. II)",Timothy Bailey,Matt Selman,2017-01-15,WABF04,6.90,28,NaN,NaN
697,712,6,"""A Serious Flanders"" (Parts 1)""",Debbie Bruce Mahan,Cesar Mazariegos,2021-11-07,QABF21,3.47,33,NaN,NaN
698,713,7,"""A Serious Flanders"" (Parts 2)""",Matthew Faughnan,Cesar Mazariegos,2021-11-14,QABF22,1.66,33,NaN,NaN


In [43]:
df['Original air date'].unique()

array(['1998-08-23', '1998-09-20', '1998-09-27', '1998-10-25',
       '1998-11-08', '1998-11-15', '1998-11-22', '1998-12-06',
       '1998-12-20', '1999-01-10', '1999-01-17', '1999-01-31',
       '1999-02-07', '1999-02-14', '1999-02-21', '1999-02-28',
       '1999-03-28', '1999-04-04', '1999-04-11', '1999-04-25',
       '1999-05-02', '1999-05-09', '1999-05-16', '1994-09-04',
       '1994-09-11', '1994-09-25', '1994-10-02', '1994-10-09',
       '1994-10-30', '1994-11-06', '1994-11-13', '1994-11-27',
       '1994-12-04', '1994-12-18', '1995-01-08', '1995-01-22',
       '1995-02-05', '1995-02-12', '1995-02-19', '1995-02-26',
       '1995-03-05', '1995-03-19', '1995-04-09', '1995-04-16',
       '1995-04-30', '1995-05-07', '1995-05-14', '1995-05-21',
       '2014-09-28', '2014-10-05', '2014-10-12', '2014-10-19',
       '2014-11-02', '2014-11-09', '2014-11-16', '2014-11-23',
       '2014-12-07', '2015-01-04', '2015-01-11', '2015-01-25',
       '2015-02-08', '2015-02-15', '2015-03-01', '2015-

In [44]:
df['No. Season'].isna().sum()

0

In [45]:
df['No.overall'].isna().sum()

0

In [46]:
df['No. inseason'].isna().sum()

0

In [47]:
df['Original air date'].isna().sum()

0

## Write cleaned data to Json file

In [48]:
df.to_json(PATH_FOLDER_WRITE+'dataset_DDB.json', orient='records', indent=4)